In [ ]:
import pandas as pd
import os

import plotly.graph_objects as go
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)


---
# <center><b>1. 파일 불러오기</b></center>
---


In [ ]:
# 파일 읽기
os.chdir('C:\민원분석참조모델\data\결과')
df_list = list(os.listdir(os.getcwd()))
index = 0
df_ = pd.read_csv(df_list[index], encoding='euc-kr')

In [ ]:
df = df_[['민원등록일자', '민원등록일시']]
df

---
# <center><b>2. 데이터 전처리</b></center>
---

In [ ]:
# 민원등록일자별 민원 빈도
df = df.groupby('민원등록일자').count()
df

In [ ]:
# 2019년 민원 빈도
# 2019년은 2월 29일이 없으므로 0으로 추가
df_2019 = df.loc[:'2019-12-31']
df_2019.rename(columns={'민원등록일시': '2019년'}, inplace=True)
df_2019.loc['2019-02-29'] = 0
df_2019.reset_index(inplace=True)
df_2019 = df_2019[['2019년']]
df_2019

In [ ]:
# 2020년 민원 빈도
df_2020 = df.loc['2020-01-01':'2020-12-31']
df_2020.rename(columns={'민원등록일시': '2020년'}, inplace=True)
df_2020.reset_index(inplace=True)
df_2020 = df_2020[['2020년']]
df_2020


In [ ]:
# 날짜 컬럼 생성
df_merged = pd.DataFrame({'날짜': pd.date_range('2020-01-01', '2020-12-31').strftime('%m/%d')})
df_merged

In [ ]:
# 빈도수 컬럼 추가
df_merged = pd.concat([df_merged, df_2019], axis=1)
df_merged = pd.concat([df_merged, df_2020], axis=1)
df_merged

In [ ]:
# 파일 저장
df_merged.to_csv(f'../../result/6.민원등록일자/민원등록일자_빈도_{df_list[index]}', encoding='euc-kr', index=False)

In [ ]:
# 날짜 인덱스화
df_merged.set_index('날짜', inplace=True)
df_merged

In [ ]:
# 월별
df_jan = df_merged.loc[:'01/31']
df_feb = df_merged.loc['02/01':'02/29']
df_mar = df_merged.loc['03/01':'03/31']
df_apr = df_merged.loc['04/01':'04/30']
df_may = df_merged.loc['05/01':'05/31']
df_jun = df_merged.loc['06/01':'06/30']
df_jul = df_merged.loc['07/01':'07/31']
df_aug = df_merged.loc['08/01':'08/31']
df_sep = df_merged.loc['09/01':'09/30']
df_oct = df_merged.loc['10/01':'10/31']
df_nov = df_merged.loc['11/01':'11/30']
df_dec = df_merged.loc['12/01':'12/31']


In [ ]:
# 월별 총 빈도
months = [
    df_jan, df_feb, df_mar, df_apr, df_may, df_jun, df_jul,
    df_aug, df_sep, df_oct, df_nov, df_dec
]

month = pd.DataFrame(columns=['2019년', '2020년'])
month['2019년'] = [i.sum()[0] for i in months]
month['2020년'] = [i.sum()[1] for i in months]
month.index = [
    '1월', '2월', '3월', '4월', '5월', '6월',
    '7월', '8월', '9월', '10월', '11월', '12월',
    ]

month

In [ ]:
# 4분기
month_1 = month[:'3월']
month_2 = month['4월':'6월']
month_3 = month['7월':'9월']
month_4 = month['10월':'12월']

months_fours = [month_1, month_2, month_3, month_4]

month_four = pd.DataFrame(columns=['2019년', '2020년'])
month_four['2019년'] = [i.sum()[0] for i in months_fours]
month_four['2020년'] = [i.sum()[1] for i in months_fours]
month_four.index = ['1분기', '2분기', '3분기', '4분기']

month_four

In [ ]:
# 계절별
spring = month['3월':'5월']
summer = month['6월':'8월']
autumn = month['9월':'11월']
winter = month.loc[['1월', '2월', '12월']]

seasons = [spring, summer, autumn, winter]

season = pd.DataFrame(columns=['2019년', '2020년'])
season['2019년'] = [i.sum()[0] for i in seasons]
season['2020년'] = [i.sum()[1] for i in seasons]
season.index = ['봄', '여름', '가을', '겨울']

season

In [ ]:
month

---
# <center><b>3. 그래프 시각화</b></center>
---

In [ ]:
# 그래프 레이아웃
layout = {
    'title': {
        'text': '<b>연도별 민원 빈도 수</b>',
        'font': {
            'size': 20
        },
        'x': 0.5
    },
    'legend': {
        'title': {
            'text': '연도'
        }
    },
    'showlegend': True,
    'xaxis': {
        'title': '날짜',
        'tickangle': -45
    },
    'yaxis': {
        'title': '민원 빈도 수',
        'showticklabels': True,
        'dtick': 50  # 그 틱의 단위, 1마다 표시
    },
    'autosize': False,
    'width': 1000,
    'height': 400,
    
}


In [ ]:
# 연도별
fig = go.Figure()
fig.add_trace(
    go.Line(x=df_merged.index, y=df_merged['2019년'],
    name='2019년')
)
fig.add_trace(
    go.Line(x=df_merged.index, y=df_merged['2020년'],
    name='2020년')
)

fig.update_layout(layout)
fig.show()


In [ ]:
# 월별 총 빈도

layout_mon = {
    'title': {
        'text': '<b>월별 민원 빈도 수</b>',
        'font': {
            'size': 20
        },
        'x': 0.5
    },
    'legend': {
        'title': {
            'text': '연도'
        }
    },
    'showlegend': True,
    'xaxis': {
        'title': '월별',
    },
    'yaxis': {
        'title': '민원 수',
        'showticklabels': True,
    },
}

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=month.index, y=month['2019년'], name='2019년',
        text = month['2019년'],
        textposition='inside',
    )
)
fig.add_trace(
    go.Bar(
        x=month.index, y=month['2020년'], name='2020년',
        text=month['2020년'],
        textposition='inside',
        marker={'color': 'crimson'}
    )
)

fig.update_layout(layout_mon)

fig.show()


In [ ]:
# 월별 총 빈도

layout_four = {
    'title': {
        'text': '<b>분기별 민원 빈도 수</b>',
        'font': {
            'size': 20
        },
        'x': 0.5
    },
    'legend': {
        'title': {
            'text': '연도'
        }
    },
    'showlegend': True,
    'xaxis': {
        'title': '분기',
    },
    'yaxis': {
        'title': '민원 수',
        'showticklabels': True,
    },
}

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=month_four.index, y=month_four['2019년'], name='2019년',
        text=month_four['2019년'],
        mode='lines+markers+text',
        textposition='top center',
    )
)
fig.add_trace(
    go.Scatter(
        x=month_four.index, y=month_four['2020년'], name='2020년',
        text=month_four['2020년'],
        textposition='top center',
        mode='lines+markers+text',
    )
)

fig.update_layout(layout_four)

fig.show()


In [ ]:
# 계절별 민원 빈도

layout_season = {
    'title': {
        'text': '<b>게절별 민원 빈도 수</b>',
        'font': {
            'size': 20
        },
        'x': 0.5
    },
    'legend': {
        'title': {
            'text': '연도'
        }
    },
    'showlegend': True,
    'xaxis': {
        'title': '계절',
    },
    'yaxis': {
        'title': '민원 수',
        'showticklabels': True,
    },
}

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=season.index, y=season['2019년'], name='2019년',
        text=season['2019년'],
        mode='lines+markers+text',
        textposition='top center',
    )
)
fig.add_trace(
    go.Scatter(
        x=season.index, y=season['2020년'], name='2020년',
        text=season['2020년'],
        mode='lines+markers+text',
        textposition='top center',
    )
)

fig.update_layout(layout_season)

fig.show()


In [ ]:
# 1월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_jan.index,
        y=df_jan['2020년'],
        name = '2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_jan.index,
        y=df_jan['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>1월 민원 빈도 수</b>',
    },
})

fig1.show()


In [ ]:
# 2월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_feb.index,
        y=df_feb['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_feb.index,
        y=df_feb['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>2월 민원 빈도 수</b>',
    },
})

fig1.show()


In [ ]:
# 3월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_mar.index,
        y=df_mar['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_mar.index,
        y=df_mar['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>3월 민원 빈도 수</b>',
    },
})

fig1.show()


In [ ]:
# 4월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_apr.index,
        y=df_apr['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_apr.index,
        y=df_apr['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>4월 민원 빈도 수</b>',
    },
})

fig1.show()


In [ ]:
# 5월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_may.index,
        y=df_may['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_may.index,
        y=df_may['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>5월 민원 빈도 수</b>',
    },
})


fig1.show()


In [ ]:
# 6월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_jun.index,
        y=df_jun['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_jun.index,
        y=df_jun['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>6월 민원 빈도 수</b>',
    },
})


fig1.show()


In [ ]:
df_jun.sum()

In [ ]:
# 7월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_jul.index,
        y=df_jul['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_jul.index,
        y=df_jul['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>7월 민원 빈도 수</b>',
    },
})

fig1.show()


In [ ]:
df_jul.sum()

In [ ]:
# 8월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_aug.index,
        y=df_aug['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_aug.index,
        y=df_aug['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>8월 민원 빈도 수</b>',
    },
})


fig1.show()


In [ ]:
df_aug.sum()

In [ ]:
# 9월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_sep.index,
        y=df_sep['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_sep.index,
        y=df_sep['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>9월 민원 빈도 수</b>',
    },
})


fig1.show()


In [ ]:
df_sep.sum()

In [ ]:
# 10월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_oct.index,
        y=df_oct['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_oct.index,
        y=df_oct['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>10월 민원 빈도 수</b>',
    },
})


fig1.show()


In [ ]:
df_sep.sum()

In [ ]:
# 11월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_nov.index,
        y=df_nov['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_nov.index,
        y=df_nov['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>11월 민원 빈도 수</b>',
    },
})


fig1.show()


In [ ]:
df_nov.sum()

In [ ]:
# 12월
fig1 = go.Figure()

fig1.add_trace(
    go.Line(
        x=df_dec.index,
        y=df_dec['2020년'],
        name='2020년',
        fill='tozeroy'
    )
)

fig1.add_trace(
    go.Line(
        x=df_dec.index,
        y=df_dec['2019년'],
        name='2019년',
        fill='tozeroy'
    )
)

fig1.update_layout(layout)
fig1.update_layout({
    'title': {
        'text': '<b>12월 민원 빈도 수</b>',
    },
})


fig1.show()


In [ ]:
df_may.sum()